<a href="https://colab.research.google.com/github/arushijha03/ParlayProject/blob/main/business_details_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import pandas as pd
import numpy as np
import json
from datetime import date

In [125]:
#inquires = pd.read_csv('/content/inquiries.csv')
#acquisition_details = pd.read_csv('/content/acquisition_details.csv')
business_details = pd.read_csv('/content/business_details.csv')
#external_data = pd.read_csv('/content/external_data.csv')
#files = pd.read_csv('/content/files.csv')
#individual_details = pd.read_csv('/content/individual_details.csv')
#inquiry_events = pd.read_csv('/content/inquiry_events.csv')
#loan_details = pd.read_csv('/content/loan_details.csv')
#users = pd.read_csv('/content/users.csv')

Preliminary cleaning

In [126]:
#inquires.info()

In [127]:
#inquires.head()

In [128]:
#inquires.drop(columns=['first_name','last_name','organization_id','updated_at','archived_at'],axis=1)

In [129]:
business_details.head()

,tax_account_connected,business_name,business_type,business_city,business_state,business_start_date,business_description,naics_code,fulltime_employees_count,fulltime_employees_annual_payroll,...,updated_at,open_banking_connected,business_address,business_zip_code,business_phone_number,is_business_address_same_as_individual,is_business_phone_number_same_as_individual,position_at_business,percent_ownership,owner_roles
0,t,Sanford and Sons,LLC,Lawton,SC,2022-10-24 07:38:20.846+00,Ambitus administratio aggredior decretum odio ...,924120,344,3210559396,...,NaN,f,21008 Runte Ramp,22096.0,1.817972e+10,f,f,Global Division Coordinator,98.50,"{authorizedSigner,beneficialOwner,controlPerson}"
1,f,"Thiel, Windler and Strosin",Nonprofit organization,East Bill,NM,2018-05-23 07:56:56.974+00,Reiciendis molestiae abutor textus sordeo cons...,236116,38,133155059,...,NaN,f,32365 Loraine Corners,23923.0,1.431356e+10,f,f,Chief Implementation Coordinator,88.58,"{authorizedSigner,beneficialOwner,controlPerson}"
2,t,Mohr LLC,Joint venture,Lanceport,NY,2018-02-27 22:50:07.374+00,Numquam ambulo theca comitatus thermae solvo c...,813990,394,1625390855,...,NaN,f,911 McClure Roads,5728.0,1.981427e+10,f,f,National Markets Specialist,35.04,"{authorizedSigner,beneficialOwner,controlPerson}"
3,f,Jenkins and Sons,S Corporation,NaN,NaN,NaN,NaN,541213,23,154540795,...,NaN,f,NaN,NaN,NaN,f,f,NaN,82.96,"{authorizedSigner,beneficialOwner,controlPerson}"
4,t,Buckridge Group,Nonprofit organization,NaN,NaN,NaN,NaN,313230,260,1429365080,...,NaN,f,NaN,NaN,NaN,f,f,NaN,78.93,"{authorizedSigner,beneficialOwner,controlPerson}"


In [130]:
business_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 50 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   tax_account_connected                        87 non-null     object 
 1   business_name                                87 non-null     object 
 2   business_type                                87 non-null     object 
 3   business_city                                40 non-null     object 
 4   business_state                               40 non-null     object 
 5   business_start_date                          40 non-null     object 
 6   business_description                         40 non-null     object 
 7   naics_code                                   87 non-null     int64  
 8   fulltime_employees_count                     87 non-null     int64  
 9   fulltime_employees_annual_payroll            87 non-null     int64  
 10  ever

In [131]:
business_details.drop(columns=['business_name','business_city','business_state','business_description','bankruptcy_description','lawsuits_description','incorporation_city','incorporation_state','missing_tax_returns_description','missing_bank_statements_description','business_address','business_zip_code','business_phone_number','updated_at','owners','owner_roles','position_at_business','user_id','organization_id','id'],axis=1,inplace=True)

In [132]:
#reformat date columns to YYYY-MM-DD
business_details['business_start_date'] = pd.to_datetime(business_details['business_start_date'])
business_details['created_at'] = pd.to_datetime(business_details['created_at'])
# Extract only the date (YYYY-MM-DD)
business_details['business_start_date'] = business_details['business_start_date'].dt.date
business_details['created_at'] = business_details['created_at'].dt.date
#Fill NAN dates with dummy date
fallback = date(1800, 1, 1)
business_details['business_start_date'] = business_details['business_start_date'].fillna(fallback)
business_details['created_at'] = business_details['created_at'].fillna(fallback)

In [133]:
#business-ein to t/f if exists/not
business_details['business_ein'] = business_details['business_ein'].notna().map({True: 't', False: 'f'})

In [134]:
#Converting t/f into binary 1/0
cols = ['tax_account_connected', 'ever_filed_bankruptcy', 'ongoing_lawsuits','is_only_owner','has_no_debts','has_no_credit_debts','credit_check_consent','is_missing_tax_returns','is_missing_bank_statements','meta_status','open_banking_connected','is_business_address_same_as_individual','is_business_phone_number_same_as_individual', 'business_ein']  # t/f string columns

business_details[cols] = (
    business_details[cols]
      .apply(lambda s: s.str.strip().str.lower())
      .replace({'t': 1, 'f': 0})
      .astype('Int8')
)


/tmp/ipython-input-604354310.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({'t': 1, 'f': 0})


In [135]:
#Two jsons ->credit_debts and debts
import pandas as pd
import json

def parse_json_list(x):
    """Accepts list, JSON string, or null; returns list of dicts (or [])."""
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    s = str(x).strip()
    if not s:
        return []
    try:
        obj = json.loads(s)
        return obj if isinstance(obj, list) else []
    except Exception:
        return []

debt_rename = {
    "originalAmount": "debt_amount",
    "interestRate": "interest",
    "currentBalance": "current_balance",
    "monthlyPayment": "monthly_payment",
    "originalDate": "orig_date",
    "termOrMaturationDate": "maturity_date",
}


def build_debt_info(business_details):
    tmp = business_details[["inquiry_id", "debts"]].copy()
    tmp["debts_parsed"] = tmp["debts"].apply(parse_json_list)

    exploded = tmp.explode("debts_parsed", ignore_index=False)
    exploded = exploded[exploded["debts_parsed"].notna()]

    if exploded.empty:
        return pd.DataFrame(columns=["inquiry_id"] + list(debt_rename.values()) + ["id", "name"])

    debt_info = pd.concat(
        [exploded[["inquiry_id"]].reset_index(drop=True),
         pd.json_normalize(exploded["debts_parsed"]).reset_index(drop=True)],
        axis=1
    ).rename(columns=debt_rename)

    return debt_info

def build_credit_debt_info(business_details):
    tmp = business_details[["inquiry_id", "credit_debts"]].copy()
    tmp["credit_parsed"] = tmp["credit_debts"].apply(parse_json_list)

    exploded = tmp.explode("credit_parsed", ignore_index=False)
    exploded = exploded[exploded["credit_parsed"].notna()]

    if exploded.empty:
        return pd.DataFrame(columns=["inquiry_id", "credit_id", "credit_name", "credit_balance"])

    credit_debt_info = pd.concat(
        [exploded[["inquiry_id"]].reset_index(drop=True),
         pd.json_normalize(exploded["credit_parsed"]).reset_index(drop=True)],
        axis=1
    ).rename(columns={"id": "credit_id", "name": "credit_name", "balance": "credit_balance"})

    return credit_debt_info


debt_info = build_debt_info(business_details)
credit_debt_info = build_credit_debt_info(business_details)

In [136]:
debt_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   inquiry_id       220 non-null    object 
 1   id               220 non-null    object 
 2   name             220 non-null    object 
 3   interest         220 non-null    float64
 4   orig_date        220 non-null    object 
 5   current_balance  220 non-null    int64  
 6   monthly_payment  220 non-null    int64  
 7   debt_amount      220 non-null    int64  
 8   maturity_date    220 non-null    object 
dtypes: float64(1), int64(3), object(5)
memory usage: 15.6+ KB


In [137]:
debt_info.head()

,inquiry_id,id,name,interest,orig_date,current_balance,monthly_payment,debt_amount,maturity_date
0,b795682e-d22f-4f81-ae08-d465d2b3e647,2d1f7039-bdfc-4aa8-bc56-9e30aacaef10,Amazon Business Net-30 Account,8.532305,2016-09-20T01:58:19.552Z,15717497,275479,47854301,2032-10-12T17:38:56.728Z
1,b795682e-d22f-4f81-ae08-d465d2b3e647,1de083d2-5ee3-4877-864a-c961ecd6a6b7,CNH Industrial Equipment Financing,8.126510,2023-06-29T00:45:59.472Z,20596027,997646,22778190,2026-05-27T05:22:00.149Z
2,b795682e-d22f-4f81-ae08-d465d2b3e647,6a906acb-30f8-436c-b3e8-4170e2e13511,Grainger Trade Credit (Net-60 Terms),3.169367,2022-07-10T01:09:57.351Z,9291448,218112,13324533,2027-10-16T00:07:07.545Z
3,ee406cbf-a250-4b75-85af-2e684fb8409a,f5c4b4b0-9739-4c42-9b99-b46235133059,John Deere Agricultural Equipment Loan,4.955852,2024-09-12T21:51:31.839Z,1957118,26204,3088701,2032-04-08T21:35:09.790Z
4,ee406cbf-a250-4b75-85af-2e684fb8409a,cfe25d65-bd8f-4c17-8e16-dadaa6a2a38a,Shopify Capital Revenue Advance,10.036156,2023-05-07T21:36:52.828Z,20823683,463376,39759882,2028-02-06T00:00:01.352Z


In [138]:
#dropping id and debt name
debt_info.drop(columns=['id','name','orig_date','maturity_date'],axis=1,inplace=True)

In [139]:
credit_debt_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   inquiry_id      237 non-null    object
 1   credit_id       237 non-null    object
 2   credit_name     237 non-null    object
 3   credit_balance  237 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 7.5+ KB


In [140]:
#dropping credit_id and credit_name
credit_debt_info.drop(columns=['credit_id','credit_name'],axis=1,inplace=True)

In [141]:
credit_debt_info.head()

,inquiry_id,credit_balance
0,4c534a15-b817-43d6-bc8c-38bf8a22c530,318571
1,4c534a15-b817-43d6-bc8c-38bf8a22c530,3094246
2,4c534a15-b817-43d6-bc8c-38bf8a22c530,2769472
3,b795682e-d22f-4f81-ae08-d465d2b3e647,1792427
4,b795682e-d22f-4f81-ae08-d465d2b3e647,3907729


In [142]:
def to_list_per_inquiry(df, key="inquiry_id", order_by=None, dropna_inside=False):
    sort_cols = [key] + (order_by or [])
    df_sorted = df.sort_values(sort_cols, kind="stable") if order_by else df.sort_values([key], kind="stable")

    value_cols = [c for c in df_sorted.columns if c != key]
    aggfunc = (lambda s: [x for x in s if pd.notna(x)]) if dropna_inside else list

    out = (
        df_sorted
          .groupby(key, dropna=False)[value_cols]
          .agg(aggfunc)
          .reset_index()
    )
    return out

debt_info = to_list_per_inquiry(debt_info, key="inquiry_id")
credit_debt_info = to_list_per_inquiry(credit_debt_info, key="inquiry_id")

In [143]:
#Drop debt and credit_debt columns from business_details
business_details.drop(columns=['debts','credit_debts'],axis=1,inplace=True)

In [144]:
business_details.head()

,tax_account_connected,business_type,business_start_date,naics_code,fulltime_employees_count,fulltime_employees_annual_payroll,ever_filed_bankruptcy,ongoing_lawsuits,is_only_owner,annual_revenue_year_one,...,credit_check_consent,is_missing_tax_returns,is_missing_bank_statements,inquiry_id,meta_status,created_at,open_banking_connected,is_business_address_same_as_individual,is_business_phone_number_same_as_individual,percent_ownership
0,1,LLC,2022-10-24,924120,344,3210559396,0,0,1,6421118792,...,1,1,0,4c534a15-b817-43d6-bc8c-38bf8a22c530,1,2025-08-31,0,0,0,98.50
1,0,Nonprofit organization,2018-05-23,236116,38,133155059,0,0,1,266310118,...,1,1,0,b795682e-d22f-4f81-ae08-d465d2b3e647,1,2025-09-02,0,0,0,88.58
2,1,Joint venture,2018-02-27,813990,394,1625390855,0,1,1,3250781710,...,1,1,1,ee406cbf-a250-4b75-85af-2e684fb8409a,1,2025-09-04,0,0,0,35.04
3,0,S Corporation,1800-01-01,541213,23,154540795,0,0,1,365543071,...,1,0,1,06ce7d90-bb7e-498f-baf6-b6114984b6a4,0,2025-08-22,0,0,0,82.96
4,1,Nonprofit organization,1800-01-01,313230,260,1429365080,0,0,1,2858730160,...,1,0,0,8259c7ac-cc7d-42a0-b5ed-c3170439775d,0,2025-09-11,0,0,0,78.93


In [145]:
def rename_with_table(df, table_name, key="inquiry_id", skip_if_already_prefixed=True):
    rename_map = {}
    prefix = f"{table_name}_"
    for c in df.columns:
        if c == key:
            continue
        if skip_if_already_prefixed and c.startswith(prefix):
            continue
        rename_map[c] = prefix + c
    return df.rename(columns=rename_map)


debt_info = rename_with_table(debt_info, "debt_info", key="inquiry_id")
credit_debt_info = rename_with_table(credit_debt_info, "credit_debt_info", key="inquiry_id")

business_details = rename_with_table(business_details, "business_details", key="inquiry_id")


In [146]:
# Save all three DataFrames as CSVs
business_details.to_csv("business_details_cleaned.csv", index=False)
debt_info.to_csv("debt_info.csv", index=False)
credit_debt_info.to_csv("credit_debt_info.csv", index=False)
